<a href="https://colab.research.google.com/github/Rohanrathod7/my-ml-labs/blob/main/08_Extreme_Gradient_Boosting_with_XGBoost/01_Classification_with_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Classification with XGBoost

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime as dt
# Import confusion matrix and train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, LogisticRegression, LinearRegression
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier






### Supervized Learning

**Which of these is a classification problem?**  
Given below are 4 potential machine learning problems you might encounter in the wild. Pick the one that is a classification problem.

- Given past performance of stocks and various other financial data, predicting the exact price of a given stock (Google) tomorrow.

[No - Not quite. This is an example of a regression problem, because we are predicting a continuous quantity.]



- Given a large dataset of user behaviors on a website, generating an informative segmentation of the users based on their behaviors.

*[No - There's nothing to predict here, this is an unsupervised (clustering) problem.]*


- --Predicting whether a given user will click on an ad given the ad content and metadata associated with the user.



- Given a user's past behavior on a video platform, presenting him/her with a series of recommended videos to watch next.

[No - Incorrect. This problem involves ranking entities and returning the highest ranked ones (in order) to the user.]


**Which of these is a binary classification problem?**   
Great! A classification problem involves predicting the category a given data point belongs to out of a finite set of possible categories. Depending on how many possible categories there are to predict, a classification problem can be either binary or multi-class. Let's do another quick refresher here. Your job is to pick the binary classification problem out of the following list of supervised learning problems.

- -- Predicting whether a given image contains a cat. [A binary classification problem involves picking between 2 choices.]


- Predicting the emotional valence of a sentence (Valence can be positive, negative, or neutral).


- Recommending the most tax-efficient strategy for tax filing in an automated accounting system.


- Given a list of symptoms, generating a rank-ordered list of most likely diseases.

### XGBoost

**XGBoost: Fit/Predict**  
It's time to create your first XGBoost model! As Sergey showed you in the video, you can use the scikit-learn .fit() / .predict() paradigm that you are already familiar to build your XGBoost models, as the xgboost library has a scikit-learn compatible API!

Here, you'll be working with churn data. This dataset contains imaginary data from a ride-sharing app with user behaviors over their first month of app usage in a set of imaginary cities as well as whether they used the service 5 months after sign-up. It has been pre-loaded for you into a DataFrame called churn_data - explore it in the Shell!

Your goal is to use the first month's worth of data to predict whether the app's users will remain users of the service at the 5 month mark. This is a typical setup for a churn prediction problem. To do this, you'll split the data into training and test sets, fit a small xgboost model on the training set, and evaluate its performance on the test set by computing its accuracy.

pandas and numpy have been imported as pd and np, and train_test_split has been imported from sklearn.model_selection. Additionally, the arrays for the features and the target have been created as X and y.

In [9]:
# Import xgboost
import xgboost as xgb
import pandas as pd # Import pandas

url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/08_Extreme_Gradient_Boosting_with_XGBoost/dataset/churn_data.csv"
# Read the CSV file
# The original code tried to read a feather file as a CSV, and had a UnicodeDecodeError.
# The file extension is feather, so it should be read using pd.read_feather.
# Also, the variable name was confusing, it should be spotify_population.
churn_data = pd.read_csv(url)
display(churn_data.head())


# Create arrays for the features and the target: X, y
X, y = churn_data.iloc[:,:-1], churn_data.iloc[:,-1]

# Create the training and test sets
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=123)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]
print("accuracy: %f" % (accuracy))


# Your model has an accuracy of around 74%. In Chapter 3, you'll learn about ways to fine tune your XGBoost models.
# For now, let's refresh our memories on how decision trees work.

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_inc_price,inc_pct,weekday_pct,fancy_car_user,city_Carthag,city_Harko,phone_iPhone,first_month_cat_more_1_trip,first_month_cat_no_trips,month_5_still_here
0,3.67,5.0,4.7,1.10,15.4,46.2,True,0,1,1,1,0,1
1,8.26,5.0,5.0,1.00,0.0,50.0,False,1,0,0,0,1,0
2,0.77,5.0,4.3,1.00,0.0,100.0,False,1,0,1,1,0,0
3,2.36,4.9,4.6,1.14,20.0,80.0,True,0,1,1,1,0,1
4,3.13,4.9,4.4,1.19,11.8,82.4,False,0,0,0,1,0,0


accuracy: 0.759400


**Decision trees**  
Your task in this exercise is to make a simple decision tree using scikit-learn's DecisionTreeClassifier on the breast cancer dataset that comes pre-loaded with scikit-learn.

This dataset contains numeric measurements of various dimensions of individual tumors (such as perimeter and texture) from breast biopsies and a single outcome value (the tumor is either malignant, or benign).

We've preloaded the dataset of samples (measurements) into X and the target values per tumor into y. Now, you have to split the complete dataset into training and testing sets, and then train a DecisionTreeClassifier. You'll specify a parameter called max_depth. Many other parameters can be modified within this model, and you can check all of them out here.

In [11]:
# Import the necessary modules
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# Create the training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Instantiate the classifier: dt_clf_4
dt_clf_4 = DecisionTreeClassifier(max_depth=4)

# Fit the classifier to the training set
dt_clf_4.fit(X_train, y_train)

# Predict the labels of the test set: y_pred_4
y_pred_4 = dt_clf_4.predict(X_test)

# Compute the accuracy of the predictions: accuracy
accuracy = float(np.sum(y_pred_4==y_test))/y_test.shape[0]
print("accuracy:", accuracy)


# It's now time to learn about what gives XGBoost its state-of-the-art performance: Boosting.

accuracy: 0.7257


### Boosting, Model Evaluation (DMatrix)

**Measuring accuracy**  
You'll now practice using XGBoost's learning API through its baked in cross-validation capabilities. As Sergey discussed in the previous video, XGBoost gets its lauded performance and efficiency gains by utilizing its own optimized data structure for datasets called a DMatrix.

In the previous exercise, the input datasets were converted into DMatrix data on the fly, but when you use the xgboost cv object, you have to first explicitly convert your data into a DMatrix. So, that's what you will do here before running cross-validation on churn_data.

- Create a DMatrix called churn_dmatrix from churn_data using xgb.DMatrix(). The features are available in X and the labels in y.
- Perform 3-fold cross-validation by calling xgb.cv(). dtrain is your churn_dmatrix, params is your parameter dictionary, nfold is the number of cross-validation folds (3), num_boost_round is the number of trees we want to build (5), metrics is the metric you want to compute (this will be "error", which we will convert to an accuracy).

In [12]:
# Create arrays for the features and the target: X, y
X, y = churn_data.iloc[:,:-1], churn_data.iloc[:,-1]

# Create the DMatrix from X and y: churn_dmatrix
churn_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary: params
params = {"objective":"reg:logistic", "max_depth":3}

# Perform cross-validation: cv_results
cv_results = xgb.cv(dtrain=churn_dmatrix, params=params,
                  nfold=3, num_boost_round=5,
                  metrics="error", as_pandas=True, seed=123)

# Print cv_results
print(cv_results)

# Print the accuracy
print(((1-cv_results["test-error-mean"]).iloc[-1]))

#  cv_results stores the training and test mean and standard deviation of the error per boosting round (tree built) as a DataFrame.
#  From cv_results, the final round 'test-error-mean' is extracted and converted into an accuracy, where accuracy is 1-error.
#  The final accuracy of around 75% is an improvement from earlier!

   train-error-mean  train-error-std  test-error-mean  test-error-std
0           0.29625         0.017060          0.29986        0.016645
1           0.26915         0.002776          0.27164        0.002463
2           0.25611         0.002730          0.25720        0.003823
3           0.25160         0.002481          0.25546        0.001942
4           0.24764         0.002331          0.24940        0.000435
0.750600006600612


**Measuring AUC**  
Now that you've used cross-validation to compute average out-of-sample accuracy (after converting from an error), it's very easy to compute any other metric you might be interested in. All you have to do is pass it (or a list of metrics) in as an argument to the metrics parameter of xgb.cv().

Your job in this exercise is to compute another common metric used in binary classification - the area under the curve ("auc"). As before, churn_data is available in your workspace, along with the DMatrix churn_dmatrix and parameter dictionary params.

In [13]:
# Perform cross_validation: cv_results
cv_results = xgb.cv(dtrain=churn_dmatrix, params=params,
                  nfold=3, num_boost_round=5,
                  metrics="auc", as_pandas=True, seed=123)

# Print cv_results
print(cv_results)

# Print the AUC
print((cv_results["test-auc-mean"]).iloc[-1])

# An AUC of 0.84 is quite strong. As you have seen, XGBoost's learning API makes it very easy to compute any metric you may be interested in. In Chapter 3,
# you'll learn about techniques to fine-tune your XGBoost models to improve their performance even further.
# For now, it's time to learn a little about exactly when to use XGBoost.

   train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0        0.768844       0.001476       0.767757      0.002737
1        0.790677       0.006640       0.788928      0.006727
2        0.815537       0.003857       0.814085      0.005832
3        0.822564       0.001682       0.821239      0.003626
4        0.827401       0.000886       0.826085      0.002024
0.8260849495138259


**Using XGBoost**  
XGBoost is a powerful library that scales very well to many samples and works for a variety of supervised learning problems. That said, as Sergey described in the video, you shouldn't always pick it as your default machine learning library when starting a new project, since there are some situations in which it is not the best option. In this exercise, your job is to consider the below examples and select the one which would be the best use of XGBoost.


- Visualizing the similarity between stocks by comparing the time series of their historical prices relative to each other.

`[This is an example of a clustering problem - there are no labels to learn from here.]`


- Predicting whether a person will develop cancer using genetic data with millions of genes, 23 examples of genomes of people that didn't develop cancer, 3 genomes of people who wound up getting cancer.

`[This would not be an ideal use of XGBoost as there are many more features than there are examples.]`

- Clustering documents into topics based on the terms used in them.

`[This is an example of a clustering problem. There are no targets and so you cannot use supervised learning.]`


- -- Predicting the likelihood that a given user will click an ad from a very large clickstream log with millions of users and their web interactions.

`[Way to end the chapter. Time to apply XGBoost to solve regression problems!]`